In [1]:
import pandas as pd
import numpy as np
import time
import re
import mibian

import warnings
warnings.filterwarnings('ignore')

import gspread
from oauth2client.service_account import ServiceAccountCredentials

from py5paisa import FivePaisaClient
from statistics import mean

from datetime import date, datetime, timedelta

In [2]:
mail = "vimarshkumar.datascience@gmail.com"
pwd ="C1K8LV38"
dob = "20010203"

#Authentication
cred={
    "APP_NAME":"5P59257949",
    "APP_SOURCE":"8146",
    "USER_ID":"EHHqYLO8bvJ",
    "PASSWORD":"d4usolgkp3V",
    "USER_KEY":"nBdpnSFPavnRFjZHDKJWVxeE3eS2Mx1P",
    "ENCRYPTION_KEY":"Gr6KN51jNjpDQrOQxpIqYkBx2wwHYD6W"
    }
client = FivePaisaClient(email=mail, passwd=pwd, dob=dob , cred = cred)
client.login()

 12:55:38 | Logged in!!


In [3]:
df = pd.read_csv(r'scripmaster-csv-format.csv')

In [4]:
def get_user_input_list(df):
    global user_input
    
    days, months, years = get_days_months_years(df)
    print("Enter 1 for single expiry tracking or 2 for two expiries tracking: ")
    while True:
        user = input()
        if user == '1' or user == '2':
            break
        else:
            print("Wrong value entered. Please enter correct value.")
    print("\n\nEnter the expiry from the above list of available expiry dates in exact format as shown: ")
    user_input_list = []
    for i in range(int(user)):
        print(f"\nPlease enter Expiry no {i+1}:")
        get_user_input(days, months, years)
        user_input = user_input.replace('-',' ')
        user_input_list.append(user_input)
        print(f"Track Expiry no. {i+1} at Sheet{i+1}")
    return user_input_list, int(user)

In [5]:
def get_days_months_years(df):
    temp_df = df[df['Name'].str.contains("BANKNIFTY", regex = False)]
    names = temp_df['Name'].values.tolist()
    filtered_names = list(filter(lambda x: len(x) > 1, list(map(lambda x: x.split(' '), names))))
    shortlisted_names = list(set(list(map(lambda x: '-'.join(x[1:4]), filtered_names))))
    print("Available expiry dates: ", shortlisted_names)
    days = list(map(lambda x: x.split('-')[0],shortlisted_names))
    months = list(map(lambda x: x.split('-')[1], shortlisted_names))
    years = list(set(list(map(lambda x: x.split('-')[2], shortlisted_names))))
    return days, months, years

In [6]:
def get_user_input(days, months, years):
    global user_input
    user_input = input()
    if user_input.split('-')[0] not in days:
        print("Wrong expiry day entered. Please enter the correct expiry day: ")
        get_user_input()
    if user_input.split('-')[1] not in months:
        print("Wrong month entered. Please enter the correct expiry month: ")
        get_user_input()
    if user_input.split('-')[2] not in years:
        print("Wrong year entered. Please enter the correct expiry year: ")
        get_user_input()

In [7]:
def get_numeric_month(val):
    if val == 'Jan':
        return '01'
    elif val == 'Feb':
        return '02'
    elif val == 'Mar':
        return '03'
    elif val == 'Apr':
        return '04'
    elif val == 'May':
        return '05'
    elif val == 'Jun':
        return '06'
    elif val == 'Jul':
        return '07'
    elif val == 'Aug':
        return '08'
    elif val == 'Sep':
        return '09'
    elif val == 'Oct':
        return '10'
    elif val == 'Nov':
        return '11'
    elif val == 'Dec':
        return '12'

In [8]:
def get_OI_and_LTP(scrip_value):
    a=[{"Exchange":"N","ExchangeType":"D","ScripCode": f"{scrip_value}"}]
    try:
#         print('Collecting LTP')
        data = client.fetch_market_depth(a)
#         print('LTP Collected')
    except Exception as e:
        print(f'Cannot get LTP data. Exception: {str(e)}')
        return None, None, True
    OI = int(data['Data'][0]['OpenInterest'])
    LTP = data['Data'][0]['LastTradedPrice']
    return OI, LTP, False

def make_the_data_list(scrips, PE = False, CE = False, ATM = False):
    list_data = list()
    for key, value in scrips.items():
        if ATM:
            pe_scrip = value[0].split('_')[0]
            ce_scrip = value[1].split('_')[0]
            PE_OI, PE_LTP, isError = get_OI_and_LTP(pe_scrip)
            if isError:
                return None, isError
            CE_OI, CE_LTP, isError = get_OI_and_LTP(ce_scrip)
            if isError:
                return None, isError
            list_data.append([CE_OI, CE_LTP,key,PE_LTP,PE_OI])
        else:
            OI, LTP, isError = get_OI_and_LTP(value)
            if isError:
                return None, isError
            if PE:
                list_data.append(['','',key,LTP,OI])
            elif CE:
                list_data.append([OI,LTP,key,'',''])
                
    return list_data, False            

In [9]:
def get_time_in_days(user_input):
    today = date.today()
    # print("Today's date:", str(today))
    now = datetime.now()
    current_time = now.strftime("%H:%M")
    # print("Current Time =", str(current_time))
    date_in_number = f"{user_input.split(' ')[2]}-{get_numeric_month(user_input.split(' ')[1])}-{user_input.split(' ')[0]}"
    expiry_time = pd.to_datetime([f"{date_in_number} 15:30"])
    latest_time = pd.to_datetime([f'{today} {current_time}'])
    return (expiry_time - latest_time).days[0]

In [10]:
def read_or_write_to_google_sheet(write_df = [], first_run = None, sheet_name = None, read_sheet = False):
    global date_val
    
    
   
    scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]

    creds = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scope)

    client = gspread.authorize(creds)
    sh = client.open("banknifty option chain")
    worksheet_list = sh.worksheets()
    worksheet_list  = list(map(lambda x: x.title, worksheet_list))
    if sheet_name not in worksheet_list:
        sh.add_worksheet(title=sheet_name, rows="1000", cols="50")
        if read_sheet:
            return []
    else:
        if read_sheet:
            sheet = client.open("banknifty option chain").worksheet(sheet_name)
            data = sheet.get_all_values()
            if len(data) > 0:
                headers = data.pop(0)
                df = pd.DataFrame(data, columns=headers)
                if df['Date'].values[0] != date_val:
                    sheet.clear()
                    return []
                return df
            else:
                return []
    if len(write_df):
        sheet = sh.worksheet(sheet_name)  # Open the spreadhseet
        sheet.update([write_df.columns.values.tolist()] + write_df.values.tolist())
        

In [11]:
def get_subdf_and_call_put_strikes(df, user_input, spot, no_of_expiries):
    sub_df = df[df['Name'].str.contains(f'BANKNIFTY {user_input}', regex = False)]
    sub_df['Strike_diff'] = abs(sub_df['StrikeRate']-spot)
    least_diff = sorted(sub_df['Strike_diff'].values.tolist())[0]
    ATM_strike = sub_df[sub_df['Strike_diff'] == least_diff]['StrikeRate'].values[0]
    sub_df = sub_df.drop(['Strike_diff'], axis = 1)
#     print("ATM strike: ", ATM_strike)

    strikes = list(set(sub_df['StrikeRate'].values.tolist()))
    strikes = sorted(strikes)
    idx = strikes.index(ATM_strike)
    if no_of_expiries == 1:
        put_strikes = strikes[idx-15:idx] 
        call_strikes = strikes[idx+1:idx+16]
    else:
        put_strikes = strikes[idx-10:idx] 
        call_strikes = strikes[idx+1:idx+11]
    return sub_df, ATM_strike, put_strikes, call_strikes

In [12]:
def get_all_scripcodes(sub_df, ATM_strike, put_strikes, call_strikes):
    put_scrips = {}
    for strike in put_strikes:
        put_scrips[strike] = sub_df[(sub_df['CpType'] == 'PE') & (sub_df['StrikeRate'] == strike)]['Scripcode'].values[0]

    call_scrips = {}
    for strike in call_strikes:
        call_scrips[strike] = sub_df[(sub_df['CpType'] == 'CE') & (sub_df['StrikeRate'] == strike)]['Scripcode'].values[0]

    atm_scrips = {}
    atm_scrips[ATM_strike] = [f"{sub_df[(sub_df['CpType'] == 'PE') & (sub_df['StrikeRate'] == ATM_strike)]['Scripcode'].values[0]}_PE",
                              f"{sub_df[(sub_df['CpType'] == 'CE') & (sub_df['StrikeRate'] == ATM_strike)]['Scripcode'].values[0]}_CE"]
    return put_scrips, call_scrips, atm_scrips

In [13]:
def create_option_df(put_scrips, call_scrips, atm_scrips, sheet_name):
    pe_list_data, isError = make_the_data_list(put_scrips, PE = True)
    if isError:
        return None, None, None, None, None, None, None, None, isError
    ce_list_data, isError = make_the_data_list(call_scrips, CE = True)
    if isError:
        return None, None, None, None, None, None, None, None, isError
    atm_list_data, isError = make_the_data_list(atm_scrips, ATM = True)
    if isError:
        return None, None, None, None, None, None, None, None, isError

    final_list = [*pe_list_data, *atm_list_data, *ce_list_data]

    option_df = pd.DataFrame(final_list, columns = ['CE_OI','CE_LTP','Strike','PE_LTP','PE_OI'])
    put_strikes = option_df[option_df['PE_OI'] != '']['Strike'].values.tolist()
    put_LTP = option_df[option_df['PE_LTP'] != '']['PE_LTP'].values.tolist()
    call_strikes = option_df[option_df['CE_OI'] != '']['Strike'].values.tolist()
    call_LTP = option_df[option_df['CE_LTP'] != '']['CE_LTP'].values.tolist()
    pe_oi_sum = sum(option_df[option_df['PE_OI'] != '']['PE_OI'].values.tolist())
    ce_oi_sum = sum(option_df[option_df['CE_OI'] != '']['CE_OI'].values.tolist())
    option_df = option_df.replace(r'^\s*$', np.NaN, regex=True)
    option_df['Weights Call'] = option_df['CE_OI'] / ce_oi_sum
    option_df['Weights Call LTP'] = option_df['CE_LTP'] * option_df['Weights Call']
    option_df['Weights Put'] = option_df['PE_OI'] / pe_oi_sum
    option_df['Weights Put LTP'] = option_df['PE_LTP'] * option_df['Weights Put']
    ce_weight_LTP_sum = round(sum(option_df[~option_df['Weights Call LTP'].isna()]['Weights Call LTP'].values.tolist()),2)
    pe_weight_LTP_sum = round(sum(option_df[~option_df['Weights Put LTP'].isna()]['Weights Put LTP'].values.tolist()),2)
    pcr = round(pe_oi_sum/ce_oi_sum, 2)
    weight_ratio = round(pe_weight_LTP_sum/ce_weight_LTP_sum, 2)
    option_df.loc[len(option_df.index)] = [ce_oi_sum, '', '','',pe_oi_sum,'',ce_weight_LTP_sum,'',pe_weight_LTP_sum]
    option_df.fillna('', inplace = True)
    while True:
        try:
            read_or_write_to_google_sheet(write_df = option_df, sheet_name = sheet_name)
            break
        except:
            print('Cannot Write to Sheet. Please check internet connection.')
    return atm_list_data, pcr, pe_oi_sum, ce_oi_sum, ce_weight_LTP_sum, pe_weight_LTP_sum, weight_ratio, option_df, False

In [14]:
def get_atm_iv(user_input, atm_scrips, spot, ATM_strike, atm_list_data):
    r = 3.56
    t = get_time_in_days(user_input)
    if t == 0:
        t = 1
    values = list(atm_scrips.values())
    for value in values[0]:
        option = value.split('_')[-1]
        strike = list(atm_scrips.keys())[0]
        if option == 'PE':
            c = mibian.BS([spot, ATM_strike, r, t], putPrice=atm_list_data[0][3])
            atm_iv_put = round(c.impliedVolatility,2)  
        elif option == 'CE':
            c = mibian.BS([spot, ATM_strike, r, t], callPrice=atm_list_data[0][1])
            atm_iv_call = round(c.impliedVolatility,2)
    return atm_iv_put, atm_iv_call      

In [15]:
def get_option_activity_detail(pe_oi_sum, ce_oi_sum, ce_weight_LTP_sum, pe_weight_LTP_sum):
    if pe_oi_sum < ce_oi_sum and pe_weight_LTP_sum > ce_weight_LTP_sum:
        return 'Call writers'
    elif pe_oi_sum < ce_oi_sum and pe_weight_LTP_sum < ce_weight_LTP_sum:
        return 'Call buyers'
    elif pe_oi_sum > ce_oi_sum and pe_weight_LTP_sum < ce_weight_LTP_sum:
        return 'Put writers'
    elif pe_oi_sum > ce_oi_sum and pe_weight_LTP_sum > ce_weight_LTP_sum:
        return 'Put buyers'
    else:
        return 'NA'

In [16]:
def calculate_std_data(data):
    global final_df

    QR1 = final_df['Ratio (OWPL/OWCL)'].quantile([0.25,0.5,0.75])[0.25]
    QR3 = final_df['Ratio (OWPL/OWCL)'].quantile([0.25,0.5,0.75])[0.75]
    IQR = round(QR3 - QR1, 2)
    median = round(np.median(final_df['Ratio (OWPL/OWCL)'].values),2)
    std = round(np.std(final_df['Ratio (OWPL/OWCL)'].values),2)
    if std < IQR:
        Upper = median + std
        Lower = median - std
    else:
        Upper = median + IQR
        Lower = median - IQR
    if data - Upper > 0:
        condition = 'Upper Breach'
        percent_breach = round(((data-Upper)/data),2)
    elif data - Lower < 0:
        condition = 'Lower Breach'
        percent_breach = round(((Lower-data)/Lower),2)
    else:
        condition = 'No Breach'
        percent_breach = ''
    return [median, IQR, std, Upper, Lower, condition, percent_breach]

In [17]:
def append_to_df(first_run, date_val, current_time, spot, ATM_strike, pe_oi_sum, atm_iv_put, ce_oi_sum, 
                atm_iv_call, pcr, ce_weight_LTP_sum, pe_weight_LTP_sum, weight_ratio, partial_data = False):
    
    blank_list = ['' for i in range(7)]
    option_activity = get_option_activity_detail(pe_oi_sum, ce_oi_sum, ce_weight_LTP_sum, pe_weight_LTP_sum)
    if not partial_data:
        df_list = [date_val, current_time, spot, ATM_strike, pe_oi_sum, atm_iv_put, ce_oi_sum, 
                     atm_iv_call, pcr, ce_weight_LTP_sum, pe_weight_LTP_sum, weight_ratio,option_activity] + blank_list
    else:
        df_list = [date_val, current_time, spot, ATM_strike, pe_oi_sum, atm_iv_put, ce_oi_sum, 
                     atm_iv_call, pcr, ce_weight_LTP_sum, pe_weight_LTP_sum, weight_ratio,option_activity]
    return df_list
        
    
def create_partial_or_complete_row(date_val,current_time, spot, ATM_strike, pe_oi_sum, atm_iv_put, ce_oi_sum, 
                                    atm_iv_call, pcr, ce_weight_LTP_sum, pe_weight_LTP_sum, weight_ratio, columns):
    global first_run, final_df
    
    if first_run:
        df_list = append_to_df(first_run, date_val, current_time, spot, ATM_strike, pe_oi_sum, atm_iv_put, ce_oi_sum, 
                atm_iv_call, pcr, ce_weight_LTP_sum, pe_weight_LTP_sum, weight_ratio)
        final_df = pd.DataFrame([df_list], columns = columns)
        first_run = False
        return final_df
    else:
        if len(final_df) < 29:
            df_list = append_to_df(first_run, date_val, current_time, spot, ATM_strike, pe_oi_sum, atm_iv_put, ce_oi_sum, 
                atm_iv_call, pcr, ce_weight_LTP_sum, pe_weight_LTP_sum, weight_ratio)
            final_df.loc[len(final_df)] = df_list
            return final_df
        else:
            df_list = append_to_df(first_run, date_val, current_time, spot, ATM_strike, pe_oi_sum, atm_iv_put, ce_oi_sum, 
                atm_iv_call, pcr, ce_weight_LTP_sum, pe_weight_LTP_sum, weight_ratio)
            final_df.loc[len(final_df)] = df_list
            std_list = calculate_std_data(weight_ratio)
            df_list = append_to_df(first_run, date_val, current_time, spot, ATM_strike, pe_oi_sum, atm_iv_put, ce_oi_sum, 
                atm_iv_call, pcr, ce_weight_LTP_sum, pe_weight_LTP_sum, weight_ratio, partial_data = True)
            df_list = df_list + std_list
            final_df.loc[len(final_df) - 1] = df_list
            return final_df

In [18]:
def check_time_difference_in_exception(previous_min):
    latest_time = datetime.now().strftime("%H:%M")
    latest_min = latest_time.split(':')[1]
    if latest_min == '00':
        latest_min = 60
    if int(latest_min) - int(previous_min) > 1:
        return True

In [ ]:
user_input_list, no_of_expiries = get_user_input_list(df)

columns = ['Date','Time','Spot','ATM','Put OI','ATM IV (Put)','Call OI','ATM IV (Call)','PCR', 'OI Weighted Call LTP(OWCL)',
           'OI Weighted Put LTP(OWPL)','Ratio (OWPL/OWCL)','Dominant Player','Median','IQR','Std Dev',
           'Upper R','Lower R','Condition','Percent Breach']

now = datetime.now()
restart = False
first_run = True # This flag is used for first iteration
final_df = []
date_val = date.today().strftime('%d/%m/%Y')

while True:
    try:
        final_df = read_or_write_to_google_sheet(sheet_name=user_input_list[0], read_sheet=True)
        if len(final_df) != 0:
            ignore_columns = ['Date','Time','Dominant Player','Condition']
            float_columns = list(filter(lambda x: x not in ignore_columns, final_df.columns.values.tolist()))
        #     dict_columns_type = {}
        #     for column in float_columns:
        #         dict_columns_type[column] = float
            for column in float_columns:
                final_df[column] = final_df[column].apply(lambda x: float(x) if x != '' else x)
            # dict_columns_type = {'Marks': float,
            #            'Fee': float}
            first_run = False
            restart = True
        break
    except:
        print('Cannot read sheet')

while True:
    if datetime.strptime(datetime.now().strftime("%H:%M"), '%H:%M') > datetime.strptime('09:15', '%H:%M'):
        break

previous_min = 99 # initialising with any arbitrary value greater than 60
latest_time = datetime.now().strftime("%H:%M")
latest_min = latest_time.split(':')[1]


while True:
    if previous_min != latest_min or first_run or restart:
        req_list_=[{"Exch":"N","ExchType":"C","Symbol":"BANKNIFTY","Expiry":"19840101","StrikePrice":"0","OptionType":"EQ"}]
        while True:
            try:
                now = datetime.now()
                current_time = now.strftime("%H:%M")
                data = client.fetch_market_feed(req_list_)
#                 print('Collected spot data')
                break
            except Exception as e:
                print(f"Cannot get spot data. Exception: {str(e)}")
                time.sleep(10)

        spot = data['Data'][0]['LastRate']
        previous_min = current_time.split(':')[1]
#         time.sleep(5)

        for user_input in user_input_list:

            sub_df, ATM_strike, put_strikes, call_strikes = get_subdf_and_call_put_strikes(df, user_input, spot, no_of_expiries)
            put_scrips, call_scrips, atm_scrips = get_all_scripcodes(sub_df, ATM_strike, put_strikes, call_strikes)
            atm_list_data, pcr, pe_oi_sum, ce_oi_sum, ce_weight_LTP_sum, pe_weight_LTP_sum, weight_ratio, option_df, isError = create_option_df(put_scrips, call_scrips, atm_scrips, f"Dataframe {user_input}")
            if isError:
                print(f'Exception raise while getting LTP data at time {current_time}. Hence record not created.')
                break
            atm_iv_put, atm_iv_call = get_atm_iv(user_input, atm_scrips, spot, ATM_strike, atm_list_data)
            final_df = create_partial_or_complete_row(date_val,current_time, spot, ATM_strike, pe_oi_sum, atm_iv_put, ce_oi_sum, 
                                                      atm_iv_call, pcr, ce_weight_LTP_sum, pe_weight_LTP_sum, weight_ratio, columns)
            time.sleep(2)
            while True:
                try:
                    read_or_write_to_google_sheet(write_df=final_df, sheet_name = user_input)
                    break
                except:
                    print('Cannot write to sheet. Please check internet connection.')
                    
        print(current_time)
        if restart:
            restart = False
    else:
        latest_time = datetime.now().strftime("%H:%M")
        latest_min = latest_time.split(':')[1]

Available expiry dates:  ['31-Mar-2022', '30-Jun-2022', '17-Mar-2022', '03-Feb-2022', '10-Mar-2022', '24-Mar-2022', '24-Feb-2022', '03-Mar-2022', '28-Apr-2022', '17-Feb-2022', '10-Feb-2022']
Enter 1 for single expiry tracking or 2 for two expiries tracking: 
1


Enter the expiry from the above list of available expiry dates in exact format as shown: 

Please enter Expiry no 1:
31-Mar-2022
Track Expiry no. 1 at Sheet1
 12:55:56 | Success
 12:55:56 | Success
 12:55:57 | Success
 12:55:57 | Success
 12:55:57 | Success
 12:55:57 | Success
 12:55:57 | Success
 12:55:57 | Success
 12:55:58 | Success
 12:55:59 | Success
 12:55:59 | Success
 12:55:59 | Success
 12:55:59 | Success
 12:55:59 | Success
 12:55:59 | Success
 12:56:00 | Success
 12:56:00 | Success
 12:56:00 | Success
 12:56:01 | Success
 12:56:02 | Success
 12:56:02 | Success
 12:56:02 | Success
 12:56:02 | Success
 12:56:02 | Success
 12:56:02 | Success
 12:56:03 | Success
 12:56:03 | Success
 12:56:03 | Success
 12:56:03 | Success